__import__ required Python libraries

In [1]:
from bs4 import BeautifulSoup
from bs4 import Comment
import datetime
import pandas as pd
import requests

__generate__ the hockey-reference url

In [2]:
date_year = 2018
stats_webpage = 'https://www.hockey-reference.com/leagues/NHL_{}.html'.format(date_year)

__create__ the soup object

In [3]:
r = requests.get(stats_webpage)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')

__extract__ the comment containing the teams table

In [4]:
team_table = soup.find(id='all_stats')
team_comment = team_table.find(string=lambda text:isinstance(text, Comment))

__convert__ the comment back into a string and __create__ another soup object from it

In [5]:
str_comment = str(team_comment)
soup2 = BeautifulSoup(str_comment, 'html.parser')

__get__ all records from the teams table

In [6]:
team_table2 = soup2.find(id='stats').find_all('td')
team_tbody = [cell.get_text() for cell in team_table2]
team_records = [tuple(team_tbody[row: row + 32]) for row in range(0, len(team_tbody), 32)]
del team_records[-1]

__make__ a list of team ids

In [7]:
team_table3 = soup2.find(id='stats').find('tbody').find_all('a')
list_team_ids = [a['href'][7:10] for a in team_table3]

__load__ the teams data into a Pandas DataFrame

In [8]:
columns_list = ['Team Name', 'Average Age', 'GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SOW', 'SOL', 'SRS', 'SOS', 'TG/G', 'EVGF', 'EVGA', 'PP', 'PPO', 'PP%', 'PPA', 'PPOA', 'PK%', 'SH', 'SHA', 'PIM/G', 'oPIM/G', 'S', 'S%', 'SA', 'SV%', 'PDO']
df_teams = pd.DataFrame.from_records(team_records, columns=columns_list)

__concatenate__ the team ids to the teams DataFrame

In [9]:
df_team_id = pd.DataFrame(list_team_ids, columns=['Team ID'])
df_concat = pd.concat([df_team_id, df_teams], axis=1)

In [10]:
df_concat.set_index('Team ID', drop=True, inplace=True)

In [11]:
del df_concat['Average Age']

In [12]:
print(df_concat.info())

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, TBL to ARI
Data columns (total 31 columns):
Team Name    31 non-null object
GP           31 non-null object
W            31 non-null object
L            31 non-null object
OL           31 non-null object
PTS          31 non-null object
PTS%         31 non-null object
GF           31 non-null object
GA           31 non-null object
SOW          31 non-null object
SOL          31 non-null object
SRS          31 non-null object
SOS          31 non-null object
TG/G         31 non-null object
EVGF         31 non-null object
EVGA         31 non-null object
PP           31 non-null object
PPO          31 non-null object
PP%          31 non-null object
PPA          31 non-null object
PPOA         31 non-null object
PK%          31 non-null object
SH           31 non-null object
SHA          31 non-null object
PIM/G        31 non-null object
oPIM/G       31 non-null object
S            31 non-null object
S%           31 non-null object
SA 